In [4]:
import cv2
import os
import numpy as np
import face_recognition as fr

In [5]:
image = fr.load_image_file("TestImage/emilia-clarke/1.jpg")
face_locations = fr.face_locations(image)

In [6]:
image = fr.load_image_file("TestImage/emilia-clarke/1.jpg")
face_landmarks_list = fr.face_landmarks(image)

In [7]:
known_image = fr.load_image_file("TestImage/2018-01-31-20-03-48-554.jpg")
unknown_image = fr.load_image_file("TestImage/1.jpg")

biden_encoding = fr.face_encodings(known_image)[0]
unknown_encoding = fr.face_encodings(unknown_image)[0]

results = fr.compare_faces([biden_encoding], unknown_encoding)

In [8]:
picture_of_me = fr.load_image_file("TestImage/2018-01-31-20-03-48-554.jpg")
my_face_encoding = fr.face_encodings(picture_of_me)[0]

In [9]:
unknown_picture = fr.load_image_file("TestImage/1.jpg")
unknown_face_encoding = fr.face_encodings(unknown_picture)[0]

In [10]:
results = fr.compare_faces([my_face_encoding], unknown_face_encoding)

if results[0] == True:
    print("It's a picture of me!")
else:
    print("It's not a picture of me!")

It's not a picture of me!


In [13]:
video_capture = cv2.VideoCapture(0)

# Load a sample picture and learn how to recognize it.
#emilia_clarke = fr.load_image_file("TestImage/emilia-clarke/1.jpg")
pornima_image = fr.load_image_file("TestImage/2018-01-31-20-03-48-554.jpg")
pornima_face_encoding = fr.face_encodings(pornima_image)[0]

# Load a second sample picture and learn how to recognize it.
nikolaj_image = fr.load_image_file("TestImage/1.jpg")
nikolaj_face_encoding = fr.face_encodings(nikolaj_image)[0]

# Create arrays of known face encodings and their names
known_face_encodings = [
    pornima_face_encoding,
    #emilia_face_encoding,
    nikolaj_face_encoding,
]
known_face_names = [
    "Pornima Bansode",
    #"Emilia Clarke",
    "Nikolaj Coster"
]
# Initialize some variables
face_locations = []
face_encodings = []
face_names = []
process_this_frame = True

while True:
    # Grab a single frame of video
    ret, frame = video_capture.read()

    # Resize frame of video to 1/4 size for faster face recognition processing
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

    # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
    rgb_small_frame = small_frame[:, :, ::-1]

    # Only process every other frame of video to save time
    if process_this_frame:
        # Find all the faces and face encodings in the current frame of video
        face_locations = fr.face_locations(rgb_small_frame)
        face_encodings = fr.face_encodings(rgb_small_frame, face_locations)

        face_names = []
        for face_encoding in face_encodings:
            # See if the face is a match for the known face(s)
            matches = fr.compare_faces(known_face_encodings, face_encoding)
            name = "Unknown"

            # # If a match was found in known_face_encodings, just use the first one.
            # if True in matches:
            #     first_match_index = matches.index(True)
            #     name = known_face_names[first_match_index]

            # Or instead, use the known face with the smallest distance to the new face
            face_distances = fr.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_face_names[best_match_index]

            face_names.append(name)

    process_this_frame = not process_this_frame
    
# Display the results
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        # Scale back up face locations since the frame we detected in was scaled to 1/4 size
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4

        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Draw a label with a name below the face
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

    # Display the resulting image
    cv2.imshow('Video', frame)

    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release handle to the webcam
video_capture.release()
cv2.destroyAllWindows()